In [0]:
%run  /brewdat-global-sales-bees/framework-bees/library_v1.0.0/main

In [0]:
dbutils.widgets.text("watermark_start_datetime", "2022-03-01 00:00:00")
watermark_start_datetime = dbutils.widgets.get("watermark_start_datetime")
print(f"watermark_start_datetime: {watermark_start_datetime}")

dbutils.widgets.text("watermark_end_datetime", "2022-03-30 23:59:59")
watermark_end_datetime = dbutils.widgets.get("watermark_end_datetime")
print(f"watermark_end_datetime: {watermark_end_datetime}")

In [0]:
df = Framework.read_landing_zone_dataframe(
    file_format=Framework.RawFileFormat.CSV,
    location=f"{Framework.LAKEHOUSE_LANDING_ROOT}/afr/customer/"
)

#display(df)

In [0]:
df = Framework.clean_column_names(df)

#display(df)

In [0]:
try:
    from pyspark.sql import functions as F

    partition_columns = ["extract_yr","extract_mth","extract_day"]

    df = (
        df
        .filter(F.col("extract_dt").between(
            F.date_format(F.lit(watermark_start_datetime), "yyyyMMdd"),
            F.date_format(F.lit(watermark_end_datetime), "yyyyMMdd"),
        ))
    )

except:
    Framework.exit_with_last_exception()

#display(df)

In [0]:
df = Framework.create_or_replace_audit_columns(df)

#display(df)

In [0]:
df = df.withColumn("extract_yr", F.substring("extract_dt",1,4))
df = df.withColumn("extract_mth", F.substring("extract_dt",1,6))
df = df.withColumnRenamed ("extract_dt", "extract_day")

#display(df)

In [0]:
schema_name = "abi_sales_afr_gh_lz"
table_name = "gh_customer_lz"

location = Framework.generate_bronze_table_location(
    source_zone="afr",
    source_sys_name="file_bees",
    source_business_domain="sales",
    table_name=table_name,
)

results = Framework.write_delta_table(
    df=df,
    location=location,
    schema_name=schema_name,
    table_name=table_name,
    load_type=Framework.LoadType.APPEND_ALL,
    partition_columns=partition_columns,
    schema_evolution_mode=Framework.SchemaEvolutionMode.ADD_NEW_COLUMNS,
)

print(results)

In [0]:
Framework.exit_with_object(results)
